In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("data/member_transaction.csv")
print(df.shape)
df.head()

(64000, 10)


,id,recency,zip_code,is_referral,channel,conversion,num_item,total_amount,avg_price,count
0,906145,10,Surburban,0,Phone,0,3.333333,31333.33333,14016.66667,3
1,184478,6,Rural,1,Web,0,4.000000,29000.00000,7250.00000,1
2,394235,7,Surburban,1,Web,0,4.000000,20500.00000,5125.00000,2
3,130152,9,Rural,1,Web,0,1.750000,20750.00000,14875.00000,4
4,940352,2,Urban,0,Web,0,3.000000,31000.00000,10333.33333,1


In [5]:
# 결측치 확인 
df.isna().sum()

id              0
recency         0
zip_code        0
is_referral     0
channel         0
conversion      0
num_item        0
total_amount    0
avg_price       0
count           0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            64000 non-null  int64  
 1   recency       64000 non-null  int64  
 2   zip_code      64000 non-null  object 
 3   is_referral   64000 non-null  int64  
 4   channel       64000 non-null  object 
 5   conversion    64000 non-null  int64  
 6   num_item      64000 non-null  float64
 7   total_amount  64000 non-null  float64
 8   avg_price     64000 non-null  float64
 9   count         64000 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 4.9+ MB


In [7]:
# 범주형 변수 리스트화 머신러닝 모델은 문자 데이터 못 읽음 ❌ (수치화 하기 위해)
cat_list = ['recency', 'zip_code', 'is_referral', 'channel', 'conversion', 'count']
cat_list

['recency', 'zip_code', 'is_referral', 'channel', 'conversion', 'count']

In [9]:
# 유니크값 카운트   유니크값이란👉 중복을 제거했을 때 남는 값들을 말해.즉, 겹치지 않는 고유한 값이야.
for cat in cat_list:
    print(df[cat].value_counts())
    print("------------------")

recency
1     8952
10    7565
2     7537
9     6441
3     5904
4     5077
6     4605
5     4510
7     4078
11    3504
8     3495
12    2332
Name: count, dtype: int64
------------------
zip_code
Surburban    28776
Urban        25661
Rural         9563
Name: count, dtype: int64
------------------
is_referral
1    32144
0    31856
Name: count, dtype: int64
------------------
channel
Web             28217
Phone           28021
Multichannel     7762
Name: count, dtype: int64
------------------
conversion
0    54606
1     9394
Name: count, dtype: int64
------------------
count
2    13356
3    13320
5    13295
4    13220
1    10809
Name: count, dtype: int64
------------------


In [10]:
# 범주형 변수 수치화
df_dummies = pd.get_dummies(df, dtype='int', drop_first=True)
df_dummies

,id,recency,is_referral,conversion,num_item,total_amount,avg_price,count,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
0,906145,10,0,0,3.333333,31333.33333,14016.666670,3,1,0,1,0
1,184478,6,1,0,4.000000,29000.00000,7250.000000,1,0,0,0,1
2,394235,7,1,0,4.000000,20500.00000,5125.000000,2,1,0,0,1
3,130152,9,1,0,1.750000,20750.00000,14875.000000,4,0,0,0,1
4,940352,2,0,0,3.000000,31000.00000,10333.333330,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
63995,838295,10,0,0,3.500000,26000.00000,8012.500000,4,0,1,0,1
63996,547316,5,1,0,1.800000,17800.00000,11300.000000,5,0,1,1,0
63997,131575,6,1,0,4.000000,30500.00000,7833.333333,2,0,1,1,0
63998,603659,1,1,0,3.200000,21600.00000,7583.333333,5,1,0,0,0


# 모델링 준비

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

In [15]:
x = df_dummies.drop(columns=['conversion', 'id'])
y = df_dummies['conversion']

In [18]:
# 학습셋, 테스트셋 분리
# 학습, 테스트 데이터 분리 / 데이터를 학습용(train) 과 테스트용(test) 으로 나누는 코드야.
# test_size=0.3👉 전체 데이터의 30%를 테스트용으로 사용
#random_state=100👉 랜덤 분할을 항상 동일하게 유지하기 위한 고정값이걸 안 쓰면코드 실행할 때마다 데이터 분할이 달라질 수 있어.
# stratify=y : 클래스 비율에 맞게 학습, 테스트 데이터 분리 / ⭐ stratify=y (중요)👉 y(타겟)의 비율을 그대로 유지하면서 나눈다는 뜻
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=100)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(51200, 10)
(12800, 10)
(51200,)
(12800,)


In [19]:
# 일단 간단하게 모델 학습 및 평가

xgb_basic = XGBClassifier()
xgb_basic.fit(x_train, y_train)  #모델 학습 (파란색이면 학습완료뜻)
xgb_basic

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [20]:
#이 코드는 XGBoost 모델 성능 평가하는 코드야.
# #의미
# xgb_basic 👉 학습이 완료된 XGBoost 모델
# x_test 👉 테스트용 데이터 (문제지)
# predict() 👉 모델이 예측한 결과 출력

y_pred = xgb_basic.predict(x_test)
print(classification_report(y_test, y_pred))

# 의미
# y_test 👉 실제 정답
# y_pred 👉 모델 예측값
# classification_report() 👉 분류 성능 요약표 출력

              precision    recall  f1-score   support

           0       0.89      0.99      0.93     10921
           1       0.75      0.26      0.38      1879

    accuracy                           0.88     12800
   macro avg       0.82      0.62      0.66     12800
weighted avg       0.87      0.88      0.85     12800



In [ ]:
# 📊 핵심 차이 정리
# 구분	Random Forest	XGBoost
# 학습방식	병렬	    순차
# 목표	평균내서안정화	오차줄이기
# 성능	   좋음	      매우좋음
# 튜닝난이도 쉬움	   어려움
# 속도	   빠름	    상대적으로 느림

# GridSearchCV

In [ ]:
# ✅ GridSearchCV란?👉 모델의 최적 하이퍼파라미터를 자동으로 찾아주는 방법
# 쉽게 말하면:“이 파라미터 저 파라미터 다 넣어보고제일 성능 좋은 조합을 찾아주는 것”

# 파라미터(Parameter)란?👉 모델을 결정하는 값(설정값) 이야.
# 쉽게 말하면:🔧 모델의 “조절 장치” (학습시킨게 파라미터?)

In [21]:
# 클래스 불균형 해결을 위한 가중치 계산(음성/양성 비율)
scale_pos_weight_value = (y_train==0).sum() / (y_train==1).sum()
scale_pos_weight_value

np.float64(5.813040585495675)

In [23]:
# xgboost 모델 설정
xgb_model = XGBClassifier(
    objective='binary:logistic',  # 이진 분류(0, 1)
    use_label_encoder=False, 
    eval_metric='aucpr',          # 불균형 데이터에 적합한 평가지표
)

#xgb_model

In [26]:
# 하이퍼파라미터 그리드 설정
#1️⃣ max_depth 👉 트리 최대 깊이3 → 단순한 모델 (과적합 적음)7 → 복잡한 모델 (과적합 위험)
#2️⃣ learning_rate👉 한 번에 얼마나 크게 학습할지 0.01 → 천천히 학습 (안정적), 0.05 → 적당, 1 → 매우 빠름 ⚠ (보통 너무 큼)
#3️⃣ n_estimators👉 트리 개수100개 → 빠름200개 → 더 정교📌 learning_rate가 낮으면👉 n_estimators를 늘려야 성능이 잘 나옴
#4️⃣ min_child_weight👉 노드를 분할하기위한 최소데이터개수기준 1 → 쉽게 분할 (과적합 위험)5 →보수적 분할 (안정적)📌 값 클수록:분할 덜 함과적합 줄어듦
#5️⃣ scale_pos_weight 👉 클래스 불균형 보정용 가중치 예:0(안팔림) = 900개1(팔림) = 100개

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 1],
    'n_estimators': [100, 200],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight': [scale_pos_weight_value] # 고정 가중치 적용
}

In [28]:
%%time
# 그리드 서치 실행 : 최적의 파라미터 조합을 찾는 것 
grid_search = GridSearchCV(
    estimator = xgb_model,       # 학습할 모델
    param_grid=param_grid,
    scoring='average_precision', # AUPRC 점수를 기준으로 최적 모델 선택
    cv=5,                  # 교차검증(cross validation) 
    verbose=1,             # 출력 메시지 양 
    n_jobs=-1    # 사용 가능한 CPU 모두 사용
)

print("Starting Grid Search...")
grid_search.fit(x_train, y_train)

Starting Grid Search...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
CPU times: total: 7.83 s
Wall time: 43.7 s


,estimator,"XGBClassifier...ree=None, ...)"
,param_grid,"{'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], 'min_child_weight': [1, 3, ...], 'n_estimators': [100, 200], ...}"
,scoring,'average_precision'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'binary:logistic'


In [29]:
# 그리드서치 결과 : 파라미터 조합 별 평가지표 점수 및 랭킹 => 표
results = pd.DataFrame(grid_search.cv_results_)

# 주요 컬럼만 보기
cols = [
    'params',
    'mean_test_score',
    'std_test_score',
    'rank_test_score'
]

results[cols].sort_values('rank_test_score').head(10)
#⃣ results[cols]👉 우리가 보고 싶은 컬럼만 선택 
#sort_values('rank_test_score')👉 성능 순위 기준으로 정렬  rank_test_score 의미: 1 = 가장 좋은 성능2 = 두 번째3 = 세 번째📌 숫자가 작을수록 성능 좋음

,params,mean_test_score,std_test_score,rank_test_score
31,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.494862,0.016201,1
35,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.494706,0.016634,2
33,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.494623,0.016530,3
29,"{'learning_rate': 0.05, 'max_depth': 5, 'min_c...",0.491714,0.015616,4
27,"{'learning_rate': 0.05, 'max_depth': 5, 'min_c...",0.491110,0.015589,5
34,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.490644,0.014874,6
32,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.490631,0.015131,7
25,"{'learning_rate': 0.05, 'max_depth': 5, 'min_c...",0.490511,0.015253,8
30,"{'learning_rate': 0.05, 'max_depth': 7, 'min_c...",0.490015,0.014901,9
17,"{'learning_rate': 0.01, 'max_depth': 7, 'min_c...",0.483338,0.014409,10


In [30]:
#베스트 파라미터 조합
#이 코드는 GridSearchCV가 찾은 “최적의 하이퍼파라미터”를 출력하는 코드야.
#1️⃣ grid_search는GridSearchCV 객체야.
#2️⃣ best_params_는👉 교차검증 평균 점수가 가장 좋았던👉 하이퍼파라미터 조합을 딕셔너리 형태로 반환해.
print(grid_search.best_params_)

{'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 200, 'scale_pos_weight': np.float64(5.813040585495675)}


In [31]:
#이 코드는 GridSearchCV에서 나온 “최고 교차검증 점수”를 보기 좋게 출력하는 코드야.
#1️⃣ f" " → f-string👉 문자열 안에 변수를 넣을 수 있는 문법
#2️⃣ grid_search.best_score_여기서 grid_search는 GridSearchCV 객체야.
#3️⃣ AUPRC Score란?
#AUPRC = Area Under Precision-Recall Curve👉 Precision-Recall 곡선 아래 면적📌 보통 클래스 불균형 데이터에서 많이 사용(예: 구매 10%, 미구매 90%)
#4️⃣ :.4f 의미 {grid_search.best_score_:.4f}👉 소수점 4자리까지만 출력

print(f"[Best CV AUPRC Score] {grid_search.best_score_:.4f}")

[Best CV AUPRC Score] 0.4949


In [32]:
# 분류 리포트
y_pred = grid_search.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.74      0.82     10921
           1       0.31      0.67      0.42      1879

    accuracy                           0.73     12800
   macro avg       0.62      0.71      0.62     12800
weighted avg       0.84      0.73      0.76     12800



In [ ]:
# 🔹 요약 비교표
# 모델	예측력	속도	해석 용이	장점
# XGBoost	★★★★★	★★☆☆☆	★★☆☆☆	최고 성능, 복잡한 데이터
# Random Forest	★★★★☆	★★★☆☆	★★★☆☆	안정적, 변수 중요도
# KNN	★★☆☆☆~★★★☆☆	★☆☆☆☆	★★★☆☆	간단, 소규모 데이터
# Decision Tree	★★☆☆☆	★★★★☆	★★★★★	직관적, 규칙 도출

In [ ]:
# 결론

# 목표가 정확도 최고 → XGBoost
# 안정적 + 변수 중요도 보고 싶다 → Random Forest
# 설명이나 소규모 데이터 → Decision Tree/KNN

In [ ]:
# # 1️⃣ XGBoost (Extreme Gradient Boosting)
# 장점
# 부스팅 방식으로 여러 약한 모델을 합쳐 예측력이 매우 높음
# 결측치 처리 가능, 특성 중요도 확인 가능
# Kaggle 같은 경진대회에서 자주 우승하는 모델

# 단점
# 파라미터 튜닝 필요 → 처음 쓰면 과적합 가능
# 학습 속도 느림 (데이터 크면)

# 추천 상황
# 변수 많고 복잡한 데이터
# 정확도 최우선 목표
# 이탈 예측, 신용 점수, 마케팅 타겟팅 등

# #2️⃣ KNN (K-Nearest Neighbors)
# 장점
# 원리 단순, 구현 쉬움
# 데이터의 패턴이 명확하면 효과적

# 단점
# 데이터 많아지면 속도 느림
# 스케일링 중요
# 이상치에 민감

# 추천 상황
# 소규모 데이터
# 직관적 결과 필요

# 3️⃣ Random Forest (랜덤포레스트)
# 장점
# 과적합 적고 안정적
# 변수 중요도 확인 가능
# 큰 데이터도 비교적 빠름

# 단점
# 부스팅(XGBoost)보다는 성능 낮음
# 해석은 어렵지만 XGBoost보다 직관적

# 추천 상황
# 데이터 많고, 안정적인 예측 필요
# 변수 중요도 보고 전략 세우기 좋음

# 4️⃣ Decision Tree (의사결정트리)
# 장점
# 직관적, 해석 용이
# 시각화 가능

# 단점
# 과적합 쉽게 발생
# 단일 트리라 예측력 낮음

# 추천 상황
# 데이터 이해, 간단한 규칙 도출용
# 분석 보고서용 설명 모델